# Extract Sample Data
This notebook will walk through the steps necessary to download the one percent data sample, extract the emebddings and metadata from each record and store them locally, as numpy arrays and json files respectively. 

### Pre-requisites: 
You will need access to the google storage account for the A20 organization (`gs://a20_dropbox`). Contact Tom (`tomdenton@google.com`) for access. 

The data sample accessed in this notebook is ~73Gb, users should make sure to have adequate network connectivity and disk memory before running the code in this notebook

**Author:** Leo Thomas - leo@developmentseed.org\
**Last updated:** 2023/06/15

In [1]:
import os
import re
import datetime
import json
import numpy as np

# Set this environment variable before loading the tensorflow 
# library to squash warning
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

# Usefull package for displaying progress bars
from tqdm.notebook import tqdm

In [2]:
DATA_DIR = os.path.abspath("./one_percent_data_sample")

### 1.0. Retrieve data from Google Storage and store locally

In [36]:
ONE_PERCENT_SAMPLE_URI = "gs://a20_dropbox/one_percent_sep_embeddings_filter"

if not os.path.exists(DATA_DIR): 
    print("Data not found locally, downloading from Google Storage...")
    os.mkdir(LOCAL_DATA_DIR)
    os.system(f"gsutil -m cp -r {ONE_PERCENT_SMAPLE_URI} {DATA_DIR}")

sample_data_filenames = [
    os.path.join(DATA_DIR, file) 
    for file in os.listdir(DATA_DIR) 
    if os.path.isfile(os.path.join(DATA_DIR, file))
]
print(f"{len(sample_data_filenames)} files to process found")

10 files to process found


### 2.0. Open dataset using tensorflow.TFRecordDataset

In [37]:
raw_dataset = tf.data.TFRecordDataset(sample_data_filenames[0])

### 2.1. Print single record
We will notice that each record is stored as a byte string

In [38]:
for raw_record in raw_dataset.take(1):
    print(raw_record)

tf.Tensor(b'\n\xaa\xe1\x12\n\xac\xe0\x12\n\tembedding\x12\x9d\xe0\x12\n\x99\xe0\x12\n\x95\xe0\x12\x08\x01\x12\r\x12\x02\x08\x0c\x12\x02\x08\x05\x12\x03\x08\x80\n"\x80\xe0\x12\x8f?\'=\xaaI\xa9=M5\xd2;#vp;\x80\xe9\x9f\xb8\xe2\'\xd0\xbb\x9a\x08\x04\xbc\x00\xcd\x91\xbasc\xf1<\x16\x81\x87=\x93\xfa\xef=B\xe3S<j3\x90=\xce\x80C\xbch\xae\x83<-\x831<\xf8Y\xc4;74\x04<\x12X\xd8;\r\x1ab:\xcaXr<\xe7\xd3\x15<\xec\x7f\x05=\xeaNv\xbc\xa0\xbd\x9e\xbdhR\x0c<\xa8J3<:n\x93=SIA\xbc\xfdH\xe7<\x1d\xd8\xea<\x00L\xe0=\x90Z\xb2\xbbxZh\xbc\xa7I\x16=b\xd0\x12>\xffND>\xce\x0c\x92\xbc\x95\xb5\x02=6\xc7\x19<r\x0e\x0b\xbc\xcbm2\xbc\xf2Z\x95\xbb\xc0\xcec\xbbm \n=\x93\r\xa6\xbc+\x11\xd1<\xda\xfck\xbb\xa8\x1b-<\x0fp}\xbc\xa3%\xc7\xbd\xe8\xb7g<j\xab\r\xbe+\xcc!\xbc\xe2\xb6\x92=o8.>\xa8Z\x82<u\xd9\xc5\xbc\xe0I\x90:\x97\x958<C\x9c\x01\xbc\x96u\xb8<\xf4dC=\xed\x15\xf8<\xf7\x19<=\xf8H\x92<\x92\xe1.<\x8d\x99z=\x95X\x1a<\xb8\x8cQ<\x8a\'N=\x03\xbc\x95;\xcb$+>\x13>l\xbcgR\xd9<\xda\x98)\xb9\xef`\xa4\xbc\xa2\xe8\x13>:\x0f\x8f\xbb\x

### 2.2. Open Record using tensorflow.train.Example 
This will give us a clearer representation of the underlying data in order to write out the extraction code. We will find that the records each have the following features: 

- **timestamp_s (integer):** the number of seconds between the start of the file and the start of the 1 min segment represented by this record
- **filename (string):** the file name
- **embedding (byte string):** the embeddings for each of the five second clips within the 1 minute segment represented by the record
- **embedding_shape (List[3] == [12, 1, 1280]):** Each record represents 12 five second clips, each having a single channel (ie: the source audio has not been processed through the separation model) and each 5 second clip corresponds to an embedding with dimension 1280


Reference for loading `tfrecords` using the `tf.train.Example` module: https://www.tensorflow.org/tutorials/load_data/tfrecord


In [39]:
for raw_record in raw_dataset.take(1): 
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "timestamp_s"
    value {
      float_list {
        value: 0
      }
    }
  }
  feature {
    key: "filename"
    value {
      bytes_list {
        value: "20200120T043000+1030_Fowlers-Gap-Wet-B_231604.flac"
      }
    }
  }
  feature {
    key: "embedding"
    value {
      bytes_list {
        value: "\010\001\022\r\022\002\010\014\022\002\010\005\022\003\010\200\n\"\200\340\022\217?\'=\252I\251=M5\322;#vp;\200\351\237\270\342\'\320\273\232\010\004\274\000\315\221\272sc\361<\026\201\207=\223\372\357=B\343S<j3\220=\316\200C\274h\256\203<-\2031<\370Y\304;74\004<\022X\330;\r\032b:\312Xr<\347\323\025<\354\177\005=\352Nv\274\240\275\236\275hR\014<\250J3<:n\223=SIA\274\375H\347<\035\330\352<\000L\340=\220Z\262\273xZh\274\247I\026=b\320\022>\377ND>\316\014\222\274\225\265\002=6\307\031<r\016\013\274\313m2\274\362Z\225\273\300\316c\273m \n=\223\r\246\274+\021\321<\332\374k\273\250\033-<\017p}\274\243%\307\275\350\267g<j\253\r\276+\314!\274\342\266\222=o8.>

### 2.3. Build a parser function
This is a function that will be mapped over each record in the dataset to produce native python data types. See above link for further explanation

In [40]:
feature_description = {
    'timestamp_s': tf.io.FixedLenFeature([], tf.float32),
    'filename': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([], tf.string),
    'embedding_shape': tf.io.FixedLenFeature([3], tf.int64)
}

# Define a function to parse the TFRecordDataset
def parse_tfrecord(example_proto):
    # Parse the features from the serialized example
    features = tf.io.parse_single_example(example_proto, feature_description)
    
    # extract embedding as 3D array of float32, from byte string 
    embedding = tf.io.parse_tensor(features["embedding"], out_type=tf.float32)
    
    return features['timestamp_s'], features["filename"], embedding, features["embedding_shape"]


In [41]:
for timestamp_s, filename, embedding, embedding_shape in raw_dataset.take(1).map(parse_tfrecord).as_numpy_iterator():
    print("Filename: ", filename)
    print("Timestamp: ", timestamp_s)
    print("Embedding_shape: ", embedding_shape)
    print("Embedding[0]: ", embedding[0])

Filename:  b'20200120T043000+1030_Fowlers-Gap-Wet-B_231604.flac'
Timestamp:  0.0
Embedding_shape:  [  12    5 1280]
Embedding[0]:  [[ 0.0408321   0.08266003  0.00641505 ...  0.02118683  0.1104449
  -0.00417373]
 [ 0.03128645  0.14746276  0.02549604 ...  0.06955769  0.00923231
  -0.02389123]
 [ 0.14228497  0.01412601  0.07481313 ...  0.01731577  0.03601275
  -0.0205441 ]
 [-0.0104753   0.03352515  0.04499019 ...  0.10699479  0.24122106
  -0.00414111]
 [-0.04448019 -0.05658907  0.02557748 ...  0.04474385  0.2329059
   0.05120574]]


### 3.0. Setup target directories for extracted data

In [42]:
# target directories
METADATA_DIR = os.path.join(DATA_DIR, "metadata")
EMBEDDINGS_DIR = os.path.join(DATA_DIR, "embeddings")

if not os.path.exists(METADATA_DIR):
    os.mkdir(METADATA_DIR)

if not os.path.exists(EMBEDDINGS_DIR):
    os.mkdir(EMBEDDINGS_DIR)

### 3.1. Extract metadata and embedding for each record in each file of the sample dataset
We will use a reg exp to extract the following values from the filename: 
- **site_id (int):** Id number of the side
- **file_datetime (timestamp):** timestamp of the file's start
- **timezone (int):** Number of hours offset from UTC at the time and place of recording. Note: in some cases the timezone was empty, which we are handling by mapping to GMT (UTC+0). This is very likely incorrect since all recording devices are located in Australia.
- **site_name (str):** Full text name of the site
- **subsite_name (str == [Wet-A|Wet-B|Dry-A|Dry-B]):** Full text subsite name
- **file_sequence_id (int)**: A unique Id for the file

In [43]:
count = 0

# The file below is generated by santizing the site and subsite names from the A20 API
# data = requests.get("https://api.acousticobservatory.org/sites?direction=asc&items=500").json()["data"]
# site_mapping = {
#    d["name"].lower().replace("  ", "-").replace(" ", "-").replace("/", "-").replace("(", "").replace(")", "").replace(":", ""):json.loads(d["notes"])["Point ID"] 
#    for d in data 
#    if d["notes"] is not None
#}

with open("site_name_to_id_mapping.json", "r") as f: 
    site_name_to_id = json.loads(f.read())

for dataset_filename in tqdm(sample_data_filenames): 
    embeddings =[]
    metadata = []
    raw_dataset = tf.data.TFRecordDataset(dataset_filename)
    for timestamp_s, filename, embedding, embedding_shape in raw_dataset.map(parse_tfrecord).as_numpy_iterator():
        [(
            #site_id, 
            file_datetime, 
            timezone, 
            site_name, 
            subsite_name, 
            file_seq_id
        )] = re.findall(
            # I'm quite proud of myself for this regex, but if anyone can see 
            # a way to simplify it, please let me know!
            #r"site_(?P<site_id>\d{4})\/(?P<datetime>\d{8}T\d{6})(?P<timezone>(?:\+\d{4})|Z)_(?P<site_name>(?:\w*|-)*)-(?P<subsite_name>(?:Wet|Dry)-(?:A|B))_(?P<file_seq_id>\d*).flac",
            r"(?P<datetime>\d{8}T\d{6})(?P<timezone>(?:\+\d{4})|Z)_(?P<site_name>(?:\w*|-)*)-(?P<subsite_name>(?:Wet|Dry)-(?:A|B))_(?P<file_seq_id>\d*).flac",
            filename.decode("utf-8")
        )
        
        # Some files have just "Z" as timezone, assume UTC in this case
        timezone = "+0000" if timezone == "Z" else timezone
        file_datetime = datetime.datetime.strptime(f"{file_datetime}{timezone}", "%Y%m%dT%H%M%S%z")
        midnight = file_datetime.replace(hour=0, minute=0, second=0)
        file_offset_since_midnight = (file_datetime - midnight).seconds
        
        # `embedding` is a 3D array with Dims [12,5,1280]
        # The first dimension, [0:11] is the distinct 5 second windows
        # within a 60 second period.
        # The second dimension [0:4] is the 5 different audio channels
        # (4 separated + 1 combined audio channel)
                
        for temporal_index, embedding_channels in enumerate(embedding):
            for channel_index, _embedding in enumerate(embedding_channels): 
                
                site_id = site_name_to_id.get(f"{site_name.lower().replace(' ', '-')}-{subsite_name.lower().replace(' ', '-')}")
                if not site_id: 
                    raise Exception(f"No site id found for site: {site_name.lower().replace(' ', '-')}-{subsite_name.lower().replace(' ', '-')}")
                  
                count +=1

                embeddings.append(_embedding)
                metadata.append({
                    "file_timestamp": int(file_datetime.timestamp()),
                    "file_seconds_since_midnight": file_offset_since_midnight,
                    "recording_offset_in_file": int(timestamp_s + (5*temporal_index)), 
                    "channel_index": channel_index,
                    "site_id": site_id,
                    "site_name": site_name, 
                    "subsite_name": subsite_name, 
                    "file_seq_id": int(file_seq_id),
                    "filename": filename.decode("utf-8")
                })
    
    # extract filename, removes extension
    stripped_filename = dataset_filename.split('/')[-1].split('.')[0]
    
    # prep filepaths for generated files
    metadata_filepath = os.path.join(METADATA_DIR, f"{stripped_filename}.json")
    numpy_filepath = os.path.join(EMBEDDINGS_DIR, f"{stripped_filename}.npy")
    
    # write metadata to disk
    with open(metadata_filepath, "w") as f: 
        f.write(json.dumps(metadata))
    
    # write embeddings to disk
    np.save(numpy_filepath, embeddings)
            
print(f"Total number of data records: {count}")

  0%|          | 0/10 [00:00<?, ?it/s]

Total number of data records: 991435
